[Reference](https://medium.com/illumination/from-zero-to-genius-how-i-built-a-blockchain-from-scratch-in-python-c558de21fa45)

# Step 1: Creating a Helper Function to Hash Data

In [1]:
# Importing required libraries
import hashlib
import json
import sys

# Defining a helper function that wraps our hashing algorithm
def hashMe(msg=""):
    if type(msg) != str:
        msg = json.dumps(msg, sort_keys=True)

    return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [2]:
transaction = {'Alice': 174, 'Bob': -174}
print(hashMe(transaction))

224621c1e16d5d087b9f8463bae50736fe810670b8445c5f0ae97d4483b36f1f


# Step 2: Generating Transactions

In [3]:
import random
random.seed(2)  # Sets a fixed seed so results are predictable every time you run the code (good for debugging and reproducibility)

def makeTransaction(maxValue = 1000):  # This will create valid transactions in the range (1, maxValue)
    sign = int(random.getrandbits(1)) * 2 - 1  # Randomly choose between -1 or 1 (whether Alice pays Bob or Bob pays Alice)
    amount = random.randint(1, maxValue)  # Randomly select the amount to be exchanged
    aliceAmount = sign * amount  # Alice's amount will be positive or negative depending on the sign
    bobAmount = -1 * aliceAmount  # Bob's amount is the opposite of Alice's, making the transaction balanced

    return {u'Alice': aliceAmount, u'Bob': bobAmount}

In [4]:
transactionBuffer = [makeTransaction() for _ in range(100)]  # Generate 100 transactions
print(transactionBuffer)  # Display the list of transactions

[{'Alice': 884, 'Bob': -884}, {'Alice': 870, 'Bob': -870}, {'Alice': -94, 'Bob': 94}, {'Alice': -370, 'Bob': 370}, {'Alice': 174, 'Bob': -174}, {'Alice': 829, 'Bob': -829}, {'Alice': 875, 'Bob': -875}, {'Alice': -258, 'Bob': 258}, {'Alice': 218, 'Bob': -218}, {'Alice': 37, 'Bob': -37}, {'Alice': 698, 'Bob': -698}, {'Alice': -442, 'Bob': 442}, {'Alice': 403, 'Bob': -403}, {'Alice': 741, 'Bob': -741}, {'Alice': 522, 'Bob': -522}, {'Alice': 381, 'Bob': -381}, {'Alice': 959, 'Bob': -959}, {'Alice': -515, 'Bob': 515}, {'Alice': -923, 'Bob': 923}, {'Alice': -892, 'Bob': 892}, {'Alice': -373, 'Bob': 373}, {'Alice': -955, 'Bob': 955}, {'Alice': -930, 'Bob': 930}, {'Alice': -434, 'Bob': 434}, {'Alice': 906, 'Bob': -906}, {'Alice': 169, 'Bob': -169}, {'Alice': 182, 'Bob': -182}, {'Alice': -237, 'Bob': 237}, {'Alice': -181, 'Bob': 181}, {'Alice': -178, 'Bob': 178}, {'Alice': -523, 'Bob': 523}, {'Alice': 369, 'Bob': -369}, {'Alice': 527, 'Bob': -527}, {'Alice': 574, 'Bob': -574}, {'Alice': -916, '

# Step 3: Updating States and Validating Transactions

In [5]:
def updateState(transaction, state):
    '''
    Inputs:
        transaction: a dictionary with keys Alice and Bob, and values the amount of money they are sending to each other
        state: a dictionary with keys Alice and Bob, and values their current balance
    Outputs:
        state: a dictionary with keys Alice and Bob, and values their new balance after the transaction
        ** Additional users will be added to state if they are not already present
    '''

    state = state.copy()  # To avoid modifying the original state

    for key in transaction.keys():
        if key in state.keys():
            state[key] += transaction[key]  # Update the balance
        else:
            state[key] = transaction[key]  # Add the new user with their balance

    return state

In [6]:
state = {'Alice': 1000, 'Bob': 1000}  # Initial balances
print(f"Initial State: {state}")

transaction = makeTransaction()  # Random transaction
print(f"Transaction: {transaction}")

newState = updateState(transaction, state)  # Apply transaction
print(f"Updated State: {newState}")

Initial State: {'Alice': 1000, 'Bob': 1000}
Transaction: {'Alice': -689, 'Bob': 689}
Updated State: {'Alice': 311, 'Bob': 1689}


In [7]:
def isValidTransaction(transaction, state):
    '''
    Inputs:
        transaction: a dictionary with keys Alice and Bob or anyone else, and values the amount of money they are sending to each other
        state: a dictionary with keys Alice and Bob or anyone else, and values their current balance
    Outputs:
        True if the transaction is valid, False otherwise
    '''
    # Rule 1: Check if the transaction is zero-sum
    if sum(transaction.values()) != 0:
        return False  # If the sum isn't zero, the transaction is invalid

    # Rule 2: Check if the sender has enough funds
    for key in transaction.keys():
        accountBalance = state[key]  # If the user is not in state, assume they have 0 balance

        # If the sender doesn't have enough funds, the transaction is invalid
        if accountBalance + transaction[key] < 0:
            return False

    return True  # If all checks pass, the transaction is valid

In [8]:
state = {'Alice': 1000, 'Bob': 1000}  # Initial balances
print(f"Initial State: {state}")

# Test different transactions
transaction1 = {'Alice': -800, 'Bob': 800}  # Alice sends 800 to Bob (valid)
transaction2 = {'Alice': 1500, 'Bob': -1500}  # Bob sends 1500 to Alice (invalid: Bob doesn't have enough funds)
transaction3 = {'Alice': -400, 'Bob': 600}  # Alice sends 400 to Bob, but Bob gets 600 (invalid: tokens are being created)
transaction4 = {'Alice': -500, 'Bob': 250, 'Lisa': 250}  # New users are valid as long as rules are preserved

# Check if transactions are valid
print(f"Transaction: {transaction1}, Valid: {isValidTransaction(transaction1, state)}")
print(f"Transaction: {transaction2}, Valid: {isValidTransaction(transaction2, state)}")
print(f"Transaction: {transaction3}, Valid: {isValidTransaction(transaction3, state)}")
print(f"Transaction: {transaction4}, Valid: {isValidTransaction(transaction4, state)}")

Initial State: {'Alice': 1000, 'Bob': 1000}
Transaction: {'Alice': -800, 'Bob': 800}, Valid: True
Transaction: {'Alice': 1500, 'Bob': -1500}, Valid: False
Transaction: {'Alice': -400, 'Bob': 600}, Valid: False


KeyError: 'Lisa'

# Step 4: Let’s Create Our Blockchain!

In [9]:
state = {'Alice': 2000, 'Bob': 2000}  # Initial state of the system (think of it as setting up the original balances in a brand-new blockchain network)

genesisBlockTransactions = [state]  # The very first block starts with the initial state as its "transaction"

genesisBlockContents = {
    'blockNumber': 0,        # This is the very first block, so block number is 0
    'parentHash': None,      # No parent because this is the genesis block
    'transactionCount': 1,   # Only one "transaction" (our initial state setup)
    'transactions': genesisBlockTransactions
}

genesisHash = hashMe(genesisBlockContents)  # Generate SHA-256 hash of the block's content (makes it tamper-proof)

genesisBlock = {'hash': genesisHash, 'contents': genesisBlockContents}  # Wrap the hash and contents together into a full block

genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)  # Serialize the block (useful for storage or transmission)

# Now we finally have the first element of our blockchain — everything else will link back to this!
chain = [genesisBlock]  # Create the blockchain list with the genesis block as the first block

In [12]:
def makeBlock(transactions, chain):
    '''
    Inputs:
        transactions: a list of transactions to include in the block
        chain: the current blockchain (a list of existing blocks)
    Outputs:
        block: a dictionary representing the new block, including its hash and its contents
    '''
    parentBlock = chain[-1]  # The last block in the chain (most recent one)
    parentHash = parentBlock['hash']  # We will reference the parent block’s hash
    blockNumber = parentBlock['contents']['blockNumber'] + 1  # New block’s number is parent's block number + 1
    transactionCount = len(transactions)  # Number of transactions in this block

    blockContents = {
        'blockNumber': blockNumber,
        'parentHash': parentHash,
        'transactionCount': transactionCount,
        'transactions': transactions
    }

    blockHash = hashMe(blockContents)  # Hash the block's content for immutability

    block = {'hash': blockHash, 'contents': blockContents}

    return block

In [13]:
blockSizeLimit = 10  # Maximum number of transactions allowed in a block

# Note: In a real blockchain, block size is usually limited by bytes, not transaction count. But for simplicity,
# we'll just limit the number of transactions here.

while len(transactionBuffer) > 0:
    bufferStartSize = len(transactionBuffer)  # How many transactions are left before starting

    transactionList = []  # This will hold the valid transactions for the next block

    # Keep collecting valid transactions until we hit the block size limit
    while (len(transactionList) < blockSizeLimit) and (len(transactionBuffer) > 0):
        newTransaction = transactionBuffer.pop()  # Get a transaction from the buffer

        transactionValidity = isValidTransaction(newTransaction, state)  # Check if it's valid
        if transactionValidity:
            transactionList.append(newTransaction)  # Add valid transaction to the block
            state = updateState(newTransaction, state)  # Update the system state
        else:
            print(f"Transaction {newTransaction} is invalid and will be discarded")
            # Invalid transactions are simply discarded
            continue

    # If we collected some valid transactions, create a new block and add it to the chain
    newBlock = makeBlock(transactionList, chain)
    chain.append(newBlock)  # Add the new block to the blockchain

    print(f"Block {newBlock['contents']['blockNumber']} created with {len(transactionList)} transactions")

Transaction {'Alice': 350, 'Bob': -350} is invalid and will be discarded
Block 1 created with 10 transactions
Transaction {'Alice': 904, 'Bob': -904} is invalid and will be discarded
Transaction {'Alice': 376, 'Bob': -376} is invalid and will be discarded
Transaction {'Alice': 520, 'Bob': -520} is invalid and will be discarded
Transaction {'Alice': 576, 'Bob': -576} is invalid and will be discarded
Transaction {'Alice': 852, 'Bob': -852} is invalid and will be discarded
Transaction {'Alice': 819, 'Bob': -819} is invalid and will be discarded
Block 2 created with 10 transactions
Block 3 created with 10 transactions
Transaction {'Alice': 816, 'Bob': -816} is invalid and will be discarded
Transaction {'Alice': 511, 'Bob': -511} is invalid and will be discarded
Transaction {'Alice': 410, 'Bob': -410} is invalid and will be discarded
Block 4 created with 10 transactions
Block 5 created with 10 transactions
Block 6 created with 10 transactions
Transaction {'Alice': -892, 'Bob': 892} is inval

In [13]:
chain[0]  # This is the genesis block, which is the first block in the chain. It has no parent and contains the initial state of the system


{'hash': '0cfb911091d3f6d330c2dff1bab497a582f6b99e91d1cca475c1639728289c17',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'transactionCount': 1,
  'transactions': [{'Alice': 2000, 'Bob': 2000}]}}

In [14]:
chain[3] # Accesing 4th block

{'hash': 'db2b43709f67e096d156ee94ecbd5f82d17d7d6d2d1f134919941b954cb159df',
 'contents': {'blockNumber': 3,
  'parentHash': '5e8f4e1a4d44713b2425b5577a11ed4a48b025b46a83265490af27fdfe443ca7',
  'transactionCount': 10,
  'transactions': [{'Alice': 856, 'Bob': -856},
   {'Alice': -835, 'Bob': 835},
   {'Alice': -964, 'Bob': 964},
   {'Alice': -675, 'Bob': 675},
   {'Alice': 468, 'Bob': -468},
   {'Alice': 571, 'Bob': -571},
   {'Alice': 744, 'Bob': -744},
   {'Alice': -360, 'Bob': 360},
   {'Alice': 925, 'Bob': -925},
   {'Alice': 466, 'Bob': -466}]}}

# Step 5: Validating Blocks and the Blockchain

In [16]:
def checkBlockHash(block):
    '''
    Inputs:
        block: a block in the blockchain
    Outputs:
        None if the hash is valid, raises an exception if the hash is invalid

    This function checks if the hash of the block is valid by comparing it to the hash of the block's contents.
    It protects the blockchain from tampering.
    If anyone changes even a single letter in a block, the hash will change, making the block invalid.
    This ensures the immutability of blockchain data.
    '''
    expectedHash = hashMe(block['contents'])  # Hash the block contents to get what the hash SHOULD be
    if expectedHash != block['hash']:
        raise Exception(f"Hash does not match for block {block['contents']['blockNumber']}. Expected {expectedHash}, got {block['hash']}")
    return

In [17]:
def checkBlockValidity(block, parent, state):
    '''
    This function validates a block before adding it to the blockchain.
    Inputs:
        block: the block to validate
        parent: the previous block (parent)
        state: the current system state
    Outputs:
        Updated system state after applying the block's transactions

    It checks:
        1. Block number is next after the parent
        2. Block points to the correct parent hash
        3. Transactions are valid
        4. Block hash is correct
    Raises an error if any check fails.
    '''
    parentNumber = parent['contents']['blockNumber']  # Get parent's block number
    parentHash = parent['hash']                       # Get parent's hash
    blockNumber = block['contents']['blockNumber']    # Get current block number

    # Check all transactions in the block
    for transaction in block['contents']['transactions']:
        if isValidTransaction(transaction, state):
            state = updateState(transaction, state)  # Update state for valid transactions
        else:
            raise Exception(f"Transaction {transaction} in block {blockNumber} is invalid")

    checkBlockHash(block)  # Verify the block's hash matches

    if blockNumber != parentNumber + 1:
        raise Exception(f"Block {blockNumber} is not the next block after {parentNumber}.")  # Ensure block numbers are sequential

    if block['contents']['parentHash'] != parentHash:
        raise Exception(f"Block {blockNumber} does not have the correct parent hash. Expected {parentHash}, got {block['contents']['parentHash']}")  # Ensure parent hash matches

    return state

In [18]:
def checkChain(chain):
    '''
    This function checks the entire blockchain for validity and consistency.
    Inputs:
        chain: list of blocks
    Outputs:
        Final system state if the chain is valid

    It checks:
        1. The chain is properly formatted
        2. The genesis block is valid
        3. Every subsequent block is valid
    '''
    if type(chain) == str:
        try:
            chain = json.loads(chain)  # If given as a string, convert to a list
            assert type(chain) == list
        except:
            return False
    elif type(chain) != list:
        return False

    state = {}  # Start with an empty state

    # Update state with genesis block transactions (we assume genesis block is valid)
    for transaction in chain[0]['contents']['transactions']:
        state = updateState(transaction, state)

    checkBlockHash(chain[0])  # Check genesis block hash

    parent = chain[0]  # Set genesis block as parent

    for block in chain[1:]:  # Validate each subsequent block
        state = checkBlockValidity(block, parent, state)
        parent = block  # Update parent for next iteration

    return state  # Return the final computed state

In [19]:
# Check the entire blockchain for validity and consistency
checkChain(chain)
# This will return the final state of the system after processing all blocks in the chain

Exception: Transaction {'Alice': -373, 'Bob': 373} in block 6 is invalid

In [20]:
# Manually tampering with a transaction in the chain to see if the checkChain function catches it
chain[1]['contents']['transactions'][0]['Alice'] = -10000  # This is an invalid transaction since Alice doesn't have enough money to send 10000 to Bob

# This will cause the checkChain function to raise an exception when we try to check the chain again
try:
    checkChain(chain)  # Check if the chain catches the invalid transaction
except Exception as e:
    print(f"Exception: {e}")

Exception: Transaction {'Alice': -10000, 'Bob': -109} in block 1 is invalid


In [21]:
# Breaking the parentHash reference in block 3
chain[1]['contents']['transactions'][0]['Alice'] = -341  # Reset the transaction to a valid one so we can check the chain again
chain[3]['contents']['parentHash'] = '1234567890'  # This is an invalid parent hash since it doesn't match the hash of the parent block

# This will cause the checkChain function to raise an exception when we try to check the chain again
try:
    checkChain(chain)  # Check if the chain detects the parent hash issue
except Exception as e:
    print(f"Exception: {e}")

Exception: Transaction {'Alice': -341, 'Bob': -109} in block 1 is invalid


In [22]:
# Breaking the block number continuity in block 4
chain[3]['contents']['parentHash'] = chain[2]['hash']  # Reset the parent hash to the correct value so we can check the chain again
chain[4]['contents']['blockNumber'] = chain[3]['contents']['blockNumber'] + 2  # This makes block 4 skip a number

# This will cause the checkChain function to raise an exception when we try to check the chain again
try:
    checkChain(chain)  # Check if the chain detects the broken block number sequence
except Exception as e:
    print(f"Exception: {e}")

Exception: Transaction {'Alice': -341, 'Bob': -109} in block 1 is invalid


# Step 7: Putting it All Together — The Final Blockchain Architecture

In [23]:
import copy

nodeBchain = copy.copy(chain)  # This is a shallow copy of the chain, so it won't affect the original chain
nodeBtransactions = [makeTransaction() for _ in range(10)]  # Create 10 transactions
newBlock = makeBlock(nodeBtransactions, nodeBchain)  # Create a new block with the transactions gathered

In [24]:
print(f"Blockchain on Node A is currently {len(chain)} blocks long")

try:
    print("New block received. Checking validity...")
    state = checkBlockValidity(newBlock, chain[-1], state)  # Check if the new block is valid (raises an error if invalid)
    chain.append(newBlock)  # Add the valid block to the chain
except:
    print("Invalid block. Ignoring and waiting for the next block...")  # Handle invalid blocks

Blockchain on Node A is currently 9 blocks long
New block received. Checking validity...
